In [1]:
import sys, os
# sys.path.insert(0,'/global/homes/b/bpb/metatlas/' )
sys.path.insert(0,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
import numpy as np
import pandas as pd
from metatlas import metatlas_objects as metob
from metatlas.helpers import dill2plots as dp
import numpy as np
import time
import copy
import dill
pd.options.display.max_colwidth = 100
pd.set_option('display.max_rows', 500)

('Metatlas live in ', '/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages/metatlas')
you're running on 3bf9cfac4637 at 172.17.0.5 


In [2]:
ref_df = pd.read_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/unique_compounds.pkl')
print ref_df.shape
ref_df.head()

(192038, 29)


,chebi_id,chebi_url,description,formula,hmdb_id,hmdb_url,img_abc_id,inchi,inchi_key,iupac_name,...,neutralized_inchi,neutralized_inchi_key,num_free_radicals,number_components,permanent_charge,pubchem_compound_id,pubchem_url,source,synonyms,wikipedia_url
0,,,,C31H52O,,,,"InChI=1S/C31H52O/c1-9-23(20(2)3)11-10-21(4)24-14-18-31(8)27-13-12-25-22(5)28(32)16-17-29(25,6)26...",UULZCTIPCNJXAT-SPSGOSLXSA-N,,...,"InChI=1S/C31H52O/c1-9-23(20(2)3)11-10-21(4)24-14-18-31(8)27-13-12-25-22(5)28(32)16-17-29(25,6)26...",UULZCTIPCNJXAT-SPSGOSLXSA-N,0,1,0,,,MetAtlas,,
1,,,,C62H88N12O17,,,,InChI=1S/C62H88N12O17/c1-27(2)42(59(85)73-23-17-19-36(73)57(83)69(13)25-38(76)71(15)48(29(5)6)61...,RWUXYUFLKBITPJ-CAANCPNXSA-N,,...,InChI=1S/C62H88N12O17/c1-27(2)42(59(85)73-23-17-19-36(73)57(83)69(13)25-38(76)71(15)48(29(5)6)61...,RWUXYUFLKBITPJ-CAANCPNXSA-N,0,1,0,,,MetAtlas,,
2,,,,C30H48O3,,,,"InChI=1S/C30H48O3/c1-19(2)9-8-10-20(3)23-17-26(33)30(18-31)22-11-12-24-27(4,5)25(32)14-15-28(24,...",SFAFFKXDWJHUSZ-IXHCEPIESA-N,,...,"InChI=1S/C30H48O3/c1-19(2)9-8-10-20(3)23-17-26(33)30(18-31)22-11-12-24-27(4,5)25(32)14-15-28(24,...",SFAFFKXDWJHUSZ-IXHCEPIESA-N,0,1,0,,,MetAtlas,,
3,,,,C8H12O8,,,,"InChI=1S/C8H12O8/c9-2-5(12)7(14)6(13)3(10)1-4(11)8(15)16/h2-3,5-7,10,12-14H,1H2,(H,15,16)/t3-,5-...",CMRFLDMXLVWBFG-SHUUEZRQSA-N,,...,"InChI=1S/C8H12O8/c9-2-5(12)7(14)6(13)3(10)1-4(11)8(15)16/h2-3,5-7,10,12-14H,1H2,(H,15,16)/t3-,5-...",CMRFLDMXLVWBFG-SHUUEZRQSA-N,0,1,0,,,MetAtlas,,
4,,,,C27H42O3,,,,"InChI=1S/C27H42O3/c1-17(6-5-7-18(2)25(29)30)22-10-11-23-21-9-8-19-16-20(28)12-14-26(19,3)24(21)1...",PSXQJZDFWDKBIP-JHIMGXENSA-N,,...,"InChI=1S/C27H42O3/c1-17(6-5-7-18(2)25(29)30)22-10-11-23-21-9-8-19-16-20(28)12-14-26(19,3)24(21)1...",PSXQJZDFWDKBIP-JHIMGXENSA-N,0,1,0,,,MetAtlas,,


In [3]:
ref_df.keys()

Index([u'chebi_id', u'chebi_url', u'description', u'formula', u'hmdb_id',
       u'hmdb_url', u'img_abc_id', u'inchi', u'inchi_key', u'iupac_name',
       u'kegg_id', u'kegg_url', u'lipidmaps_id', u'lipidmaps_url',
       u'metacyc_id', u'mono_isotopic_molecular_weight', u'name',
       u'neutralized_2d_inchi', u'neutralized_2d_inchi_key',
       u'neutralized_inchi', u'neutralized_inchi_key', u'num_free_radicals',
       u'number_components', u'permanent_charge', u'pubchem_compound_id',
       u'pubchem_url', u'source', u'synonyms', u'wikipedia_url'],
      dtype='object')

In [2]:
dp = reload(dp)
groups = dp.select_groups_for_analysis(name = '%nyogilipidsA%',
                                       most_recent = True,
                                       remove_empty = True,
                                       include_list = [], exclude_list = ['QC','Blank','extractioncontrol'])

49
(0, u'20160504_KBL_NyogiLipidsA_NEG_72hr_BD_controlBD_noglc', datetime.datetime(2016, 7, 18, 19, 26, 3))
(1, u'20160504_KBL_NyogiLipidsA_POS_24hr_BD_controlBD_noglc', datetime.datetime(2016, 7, 18, 19, 26, 28))
(2, u'20160504_KBL_NyogiLipidsA_POS_48hr_BD_treatmentBD_glc', datetime.datetime(2016, 7, 18, 19, 26, 4))
(3, u'20160504_KBL_NyogiLipidsA_POS_24hr_BD_treatmentBD_glc', datetime.datetime(2016, 7, 18, 19, 26, 5))
(4, u'20160504_KBL_NyogiLipidsA_FPS_standards', datetime.datetime(2016, 7, 18, 19, 26, 35))
(5, u'20160504_KBL_NyogiLipidsA_POS_OleicAcid_D9__', datetime.datetime(2016, 7, 18, 19, 26, 5))
(6, u'20160504_KBL_NyogiLipidsA_POS_PC_D31__', datetime.datetime(2016, 7, 18, 19, 26, 5))
(7, u'20160504_KBL_NyogiLipidsA_POS_48hr_BD_controlBD_noglc', datetime.datetime(2016, 7, 18, 19, 26, 6))
(8, u'20160504_KBL_NyogiLipidsA_POS_48hr_BG_controlBG_glcglc', datetime.datetime(2016, 7, 18, 19, 26, 7))
(9, u'20160504_KBL_NyogiLipidsA_NEG_12hr_BG_controlBG_glcglc', datetime.datetime(2016, 

# Functions

In [3]:
def dataframe_container_from_metatlas_hdf5_file(my_file):
    data_df = pd.DataFrame()
    pd_h5_file  = pd.HDFStore(my_file)
    keys = pd_h5_file.keys()
    pd_h5_file.close()
    df_container = {}
    for k in keys:
        if ('ms' in k) and not ('_mz' in k):
            new_df = pd.read_hdf(my_file,k)
            df_container[k[1:]] = new_df
    return df_container


In [4]:
import os
df = pd.DataFrame()
for group in groups:
    for my_file in group.items:
        container = dataframe_container_from_metatlas_hdf5_file(my_file.hdf5_file)
        for k in container.keys():
            if ('ms2' in k) and ('pos' in k):
                container[k]['polarity'] = 'positive'
                container[k]['hdf5_file'] = os.path.basename(my_file.hdf5_file)
                df = pd.concat([df,container[k]],axis=0)
            elif ('ms2' in k) and ('neg' in k):
                container[k]['polarity'] = 'negative'
                container[k]['hdf5_file'] = os.path.basename(my_file.hdf5_file)
                df = pd.concat([df,container[k]],axis=0)


In [5]:
df.head()

,mz,i,rt,precursor_MZ,precursor_intensity,collision_energy,polarity,hdf5_file
0,54.924725,1707.031982,0.016607,174.960007,2790502.75,20.0,negative,20160504_C18_lipid_NEG_MSMS_KBL_Qex_MR_21_72hr_BD_a_controlBD_noglc___Run44.h5
1,55.410946,1535.511230,0.016607,174.960007,2790502.75,20.0,negative,20160504_C18_lipid_NEG_MSMS_KBL_Qex_MR_21_72hr_BD_a_controlBD_noglc___Run44.h5
2,57.473019,1776.081421,0.016607,174.960007,2790502.75,20.0,negative,20160504_C18_lipid_NEG_MSMS_KBL_Qex_MR_21_72hr_BD_a_controlBD_noglc___Run44.h5
3,71.284447,1834.279419,0.016607,174.960007,2790502.75,20.0,negative,20160504_C18_lipid_NEG_MSMS_KBL_Qex_MR_21_72hr_BD_a_controlBD_noglc___Run44.h5
4,76.574112,1567.962524,0.016607,174.960007,2790502.75,20.0,negative,20160504_C18_lipid_NEG_MSMS_KBL_Qex_MR_21_72hr_BD_a_controlBD_noglc___Run44.h5


In [6]:
grouped = df.groupby(['precursor_MZ','rt','polarity','precursor_intensity','collision_energy','hdf5_file']).aggregate(lambda x: tuple(x))
grouped.mz = grouped.mz.apply(list)
grouped.i = grouped.i.apply(list)
grouped = grouped.reset_index()
grouped['spectrum'] = map(lambda x,y:(x,y),grouped['mz'],grouped['i'])
grouped['spectrum'] = grouped['spectrum'].apply(lambda x: zip(x[0],x[1]))
grouped.drop(['mz','i'], axis=1, inplace=True)


In [8]:
df.to_pickle('/scratch2/scratchdirs/bpb/dataframe_storage/df_melissa_lipids_msms_spectra.pkl')

In [9]:
print df.shape
print grouped.shape

(14131127, 8)
(623630, 7)


In [24]:
for k in grouped.keys():
    print k,'\t',getattr(grouped,k)[0]
print type(grouped.spectrum[0])

precursor_MZ 	80.0496444702
rt 	1.02968382835
polarity 	positive
precursor_intensity 	585395.125
collision_energy 	20.0
hdf5_file 	20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_19_48hr_BD_c_treatmentBD_glc___Run82.h5
spectrum 	[(53.003055572509766, 147646.359375), (53.03924560546875, 7690.60107421875), (53.998130798339844, 6578.44873046875), (54.006572723388672, 1333.9654541015625), (63.530872344970703, 1273.2967529296875), (66.886268615722656, 1213.8836669921875), (67.252754211425781, 1359.916259765625), (74.269264221191406, 1242.3798828125), (79.0421142578125, 7939.3515625), (79.054801940917969, 13506.3173828125), (80.0452880859375, 1881.0626220703125), (80.050003051757812, 49810.40234375), (81.070259094238281, 9823.8125), (92.106292724609375, 1842.271240234375)]
has_dgdg_nl 	True
<type 'list'>


In [10]:
grouped.head()

,precursor_MZ,rt,polarity,precursor_intensity,collision_energy,hdf5_file,spectrum
0,80.049644,1.029684,positive,5.853951e+05,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_19_48hr_BD_c_treatmentBD_glc___Run82.h5,"[(53.0030555725, 147646.359375), (53.0392456055, 7690.60107422), (53.9981307983, 6578.44873047),..."
1,80.049644,7.208591,positive,1.073722e+06,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_11_24hr_BD_c_treatmentBD_glc___Run85.h5,"[(50.0159378052, 1531.24108887), (51.0238189697, 10235.1875), (53.0030555725, 316263.71875), (53..."
2,80.049652,4.948564,positive,1.116939e+06,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_55_48hr_BG_c_controlBG_glcglc___Run91.h5,"[(51.0237159729, 3196.42651367), (53.0030670166, 295492.71875), (53.0394630432, 34659.96875), (5..."
3,80.049652,7.676896,positive,1.179940e+06,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_39_12hr_BG_c_controlBG_glcglc___Run97.h5,"[(51.0238075256, 13222.7382812), (52.0315895081, 1647.82775879), (52.3341140747, 1239.13586426),..."
4,80.049660,0.765142,positive,5.261536e+05,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_3_0hr_BD_c_controlBD_noglc___Run88.h5,"[(53.0030555725, 137306.203125), (53.0392875671, 8933.91601562), (53.9981498718, 7395.19238281),..."


### Scan spectra for neutral losses
* DGDG: neutral loss of 341 or 180.0634\*2-1.0078*3-15.994915
    - also DGDG with NH4 subtracted seems to be common
* MGDG: neutral loss of 179
* PG: neutral loss of 189

In [33]:
target = 236.1485
min_intensity = 1e4
ppm = 100
neutral_loss = 180.0634*2-1.0078*3-15.994915
min_precursor_intensity = 1e4
min_precursor_mz = 500
polarity = 'positive'

def search_spectrum(spectrum,target,min_intensity,ppm):
    for s in spectrum:
        if (abs(s[0] - target) < target*ppm/1e6) & (s[1] > min_intensity):
            return True
    return False

def search_neutral_loss_spectrum(row,neutral_loss,min_intensity,ppm,min_precursor_mz,min_precursor_intensity,polarity):
    if (row.precursor_MZ > min_precursor_mz) & (row.polarity == polarity) & (row.precursor_intensity > min_precursor_intensity):
        #TODO: replace with numpy or numexpr
        for s in row.spectrum:
            if (abs(s[0] - row.precursor_MZ) < row.precursor_MZ*ppm/1e6) & (s[1] > min_intensity):
                return True
    return False

# grouped['has_target_fragment'] = grouped['spectrum'].apply(search_spectrum,args = (target,min_intensity,ppm))
grouped['has_dgdg_nl'] = grouped.apply(search_neutral_loss_spectrum,
                                       axis=1,
                                       args=(neutral_loss,
                                             min_intensity,
                                             ppm,
                                             min_precursor_mz,
                                             min_precursor_intensity,
                                             polarity))


In [ ]:
# idx = (grouped.has_target_fragment) & (grouped.rt > 3.5) & (grouped.rt < 7.5) & (grouped.precursor_MZ > 500)
# print len(idx)
# df_sqdg = grouped[idx]

In [38]:
def make_unique_mzrt(df,mz_cutoff = 0.02, rt_cutoff = 0.1):
    '''
    sort the spectra in descending order
    keep only mz different by mz_cutoff
    '''
    df = df.sort_values('precursor_intensity',ascending=False)
    df_unique = pd.DataFrame()
    df_unique = df.iloc[[0]]
    for i in df.index:
        idx = abs(df.loc[i,'precursor_MZ'] - df_unique.precursor_MZ) < mz_cutoff
        if sum(idx) == 0:
            #The mz is new so add it
            df_unique = pd.concat([df_unique,df.loc[[i]]],axis = 0)
        else:
            #the mz has been seen before; so see if the rt is new
            if sum(abs(df_unique[idx].rt - df.loc[i,'rt']) < rt_cutoff)==0:
                df_unique = pd.concat([df_unique,df.loc[[i]]],axis = 0)
    return df_unique
df_unique_dgdg = make_unique_mzrt(grouped[grouped.has_dgdg_nl])

In [39]:
df_unique_dgdg.shape

(3288, 8)

In [ ]:
df_unique_dgdg.head(10)

In [41]:
df_unique_dgdg.to_csv('/global/homes/b/bpb/Downloads/dgdg_potential_hits.csv')

In [42]:
from matplotlib import pyplot as plt
%matplotlib inline

In [1]:
print(plt.style.available)

NameError: name 'plt' is not defined

In [51]:
import os

In [ ]:
outdir = '/global/homes/b/bpb/Downloads/dgdg_msms/'
for i in df_unique_dgdg.index:
    s = df_unique_dgdg.loc[i,'spectrum']
    mz = zip(*s)[0]
    intensity = zip(*s)[1]
    fig = plt.figure(figsize=(11,8.5))
    plt.style.use('seaborn-ticks')
    ax = plt.gca()
    ax.vlines(mz,intensity*0,intensity)
    ax.set_xlim(0,df_unique_dgdg.loc[i,'precursor_MZ']+5)
    ax.set_xlabel('m/z')
    ax.set_ylabel('intensity')
    sx = np.argsort(intensity)[::-1]
    labels = [1.001e9]
    for i_mz in sx:
        if np.min(np.abs(mz[i_mz] - labels)) > 0.1 and intensity[i_mz] > 0.02 * np.max(intensity):
            ax.annotate('%5.4f'%mz[i_mz], xy=(mz[i_mz], 1.01*intensity[i_mz]),rotation = 90, horizontalalignment = 'center', verticalalignment = 'left')
            labels.append(mz[i_mz])
    plt.tight_layout()
    L = plt.ylim()
    plt.ylim(L[0],L[1]*1.12)
    if not os.path.isdir(outdir):
        os.mkdir(outdir)
    fig.savefig(os.path.join(outdir,'msms_spectra_%d.png'%i))
    plt.close(fig)


Traceback (most recent call last):
  File "/global/common/edison/software/python/2.7-anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1120, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/global/common/edison/software/python/2.7-anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 301, in wrapped
    return f(*args, **kwargs)
  File "/global/common/edison/software/python/2.7-anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 364, in _fixed_getinnerframes
    lines = ulinecache.getlines(file)[start:end]
  File "/global/common/edison/software/python/2.7-anaconda/lib/python2.7/site-packages/IPython/utils/ulinecache.py", line 37, in getlines
    return [l.decode(encoding, 'replace') for l in lines]
  File "/global/common/edison/software/python/2.7-anaconda/lib/python2.7/encodings/utf_8.py", line 15, in decode
    def decode(input, errors='strict'):
KeyboardInterrupt


ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



IndexError: string index out of range

In [ ]:
1